<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA/blob/main/lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №5
# Разметка данных для решения задачи классификации.
**19 марта 2025г.**

Методы детекции анномалий оценивают отклонение от нормального многообразия. Но такая оценка не дает информации о том, какие действия следует предпринять. Задача классификации позволяет определить к какому из заранее определенного класса анномалий отнести конкретную анномалии, что в свою очередь позволяет принять адекватное решение.

In [ ]:
pip install --upgrade plotly-resampler

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(rc={'figure.figsize':(15,5)})
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import glob
import json

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler, FigureWidgetResampler

from plotly_resampler import register_plotly_resampler, unregister_plotly_resampler

pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')

In [ ]:
register_plotly_resampler(mode="auto", default_n_shown_samples=10000)

In [ ]:
plot_layout = go.Layout(
    autosize=False,
    width=1000,
    height=400,
    # xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    # yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=10, t=40, pad=4),
    showlegend = True
)

Вспомогательные функции

In [ ]:
def plot_graph():

    params_dropdown = widgets.Dropdown(
        options=description_to_kks.keys(),
        description='Сигнал',
        disabled=False,
        value=None
    )

    display(params_dropdown)
    out = widgets.Output()
    display(out)

    fig = go.Figure(layout=plot_layout)

    @out.capture()
    def params_dropdown_eventhandler(change):

        kks = description_to_kks[change.new]
        fig.add_traces(
            [
                {"x": data[kks].index, "y": data[kks].values, "name": kks, "type": "scattergl"},
            ],
        )

        clear_output()
        display(fig)

    params_dropdown.observe(params_dropdown_eventhandler, names='value')

In [ ]:
def search_widget():
    search_input = widgets.Text(placeholder="Введите текст для поиска")
    output = widgets.Output()

    def on_text_change(change):
        with output:
            output.clear_output()
            query = change["new"].lower()
            filtered_items = [f'{item} - {description_to_kks[item]}' for item in description_to_kks.keys() if query in item.lower()]
            display(filtered_items if filtered_items else "Нет совпадений")

    search_input.observe(on_text_change, names='value')
    display(search_input, output)


### Загрузка данных

In [ ]:
# import gdown
# url = "https://drive.google.com/drive/folders/1RtrAevJUYSgTbp0YUztxEBB8_VcvjgGH?usp=drive_link"
# gdown.download_folder(url, quiet=True, verify=False)

In [ ]:
parquetFileList = glob.glob(f'./option_0/*.gzip')

In [ ]:
parquetFileList

['./option_0\\file_0.gzip',
 './option_0\\file_1.gzip',
 './option_0\\file_2.gzip',
 './option_0\\file_3.gzip',
 './option_0\\file_4.gzip',
 './option_0\\file_5.gzip',
 './option_0\\file_6.gzip',
 './option_0\\file_7.gzip']

Читаем все файлы и объединяем их в общий DataFrame

In [ ]:
df_list = []

for file in tqdm(parquetFileList):
    df = pd.read_parquet(file)
    df_list.append(df)

data = pd.concat(df_list, axis=0).sort_index().ffill().drop_duplicates()
data = data.dropna()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.78it/s]


In [ ]:
print(f"Размерность датасета: {data.shape[0]} строк и {data.shape[1]} столбцов")

Размерность датасета: 1082098 строк и 31 столбцов


In [ ]:
data.head()

,GTA1.DBinPU.Aldi,GTA1.DBinPU.Alvna,GTA1.DBinPU.Alzzo,GTA1.DBinPU.Bo,GTA1.DBinPU.DlPkf,GTA1.DBinPU.DlPtgft,GTA1.DBinPU.DlPvf,GTA1.DBinPU.fi,GTA1.DBinPU.hmGTD,GTA1.DBinPU.hmTG,...,GTA1.DBinPU.Ptgpd,GTA1.DBinPU.Ptgvh,GTA1.DBinPU.Pvh,GTA1.DBinPU.Pvyhlg,GTA1.DBinPU.Qtg,GTA1.DBinPU.Tk,GTA1.DBinPU.Tn,GTA1.DBinPU.Tt,GTA1.DBinPU.Tvh1,GTA1.DBinPU.Pzad
2023-02-26 06:59:50,225.550000,-4.398000,-0.082000,101.044,113.088000,33.709000,0.200000,56.615000,442.64,444.536,...,1.834,1.921,91.455,0.667000,1794.35,263.600000,-10.524,418.80,-12.567,5850.0
2023-02-26 07:00:00,225.610769,-4.327115,-0.089769,101.044,113.151231,33.709000,0.200000,56.615000,442.64,444.536,...,1.834,1.921,91.455,0.665038,1794.35,263.600000,-10.524,418.80,-12.567,5850.0
2023-02-26 07:00:10,225.436897,-4.363724,-0.091000,101.044,113.039759,33.633586,0.200207,56.516103,442.64,444.536,...,1.834,1.921,91.455,0.667276,1794.35,263.600000,-10.524,418.80,-12.567,5850.0
2023-02-26 07:00:20,225.710833,-4.306625,-0.093000,101.044,113.053458,33.506500,0.200750,56.376000,442.64,444.536,...,1.834,1.921,91.455,0.667250,1794.35,263.600000,-10.524,418.80,-12.567,5850.0
2023-02-26 07:00:30,226.355000,-4.183118,-0.091500,101.044,113.730941,33.818529,0.200000,56.199353,442.64,444.536,...,1.834,1.921,91.455,0.676706,1800.56,263.723529,-10.524,418.85,-12.567,5850.0


Чтение файла с описанием сигналов

In [ ]:
with open(f'./option_0/description.json', 'r', encoding = "utf-8") as f:
    description = json.load(f)

Составим словарь для трактовки наименований сигналов

In [ ]:
kks_to_description = {param['real_kks']: f"{param['description']}, [{param['unit']}]"
for param in description if param['real_kks'] in data.columns}

description_to_kks = { f"{param['description']}, [{param['unit']}]": param['real_kks']
for param in description if param['real_kks'] in data.columns}

In [ ]:
# df[['GTA1.DBinPU.P', 'GTA1.DBinPU.nst']].plot(backend="plotly")

### Разметка данных

На практике размеченные наборы данных очень сложно создавать из-за редкости анномальных наблюдений, ограниченного количества экспертов, которые могут выполнять разметку, и других факторов.

Чтобы не упираться в ограничения связанные с отсутствием данных по неисправному состоянию агрегата, заменим задачу класификации дефектов на задачу класификации эксплуатационных режимов.

#### **Классы режимов для ГТУ:**
* пусковой режим
* режим останова
* режим 100% (по положению ВНА)
* режим пониженной нагрузки
* режим изменения нагрузки
* система подогрева воздуха включена
* система подогрева воздуха отключена

### Классификация на основе правил

**Разработка математических моделей на основе экспертных знаний** – это методология, при которой модель строится не на основании больших массивов данных и машинного обучения, а на основе знаний экспертов и формализованных правил. Такой подход часто используется, когда:  

1. **Недостаточно данных** – например, аварийные или нештатные режимы редки, и обучить нейросеть на таких примерах сложно.  
2. **Высокие требования к интерпретируемости** – правила, основанные на экспертных знаниях, прозрачны и объяснимы, в отличие от "черного ящика" нейросетей.  
3. **Доменная специфика** – эксперты могут использовать накопленный опыт для явного кодирования физических и логических зависимостей.  
  

Задача классификации эксплуатационных режимов может быть решена как с помощью машинного обучения, так и с помощью экспертных правил.  

**Комбинированные подходы**  
На практике часто сочетают оба метода. Например:  
1. **Гибридная модель** – экспертные правила используются для предобработки данных и фильтрации аномалий, а затем применяется машинное обучение.  
2. **Использование правил для объяснения решений модели** – например, после классификации нейросетью можно наложить интерпретируемые правила для проверки корректности.

### Анализ данных

In [ ]:
plot_graph()

Dropdown(description='Сигнал', options=('Положение дозатора топливного газа, [градус]', 'Положение ВНА ГТД, [г…

Output()

In [ ]:
search_widget()

Text(value='', placeholder='Введите текст для поиска')

Output()

#### Выделим дополнительные признаки

In [ ]:
# силовая турбина вращается с номинальной скоростью
data['full_speed'] = data['GTA1.DBinPU.nst'] > 2990

In [ ]:
# генератор синхронизирован с сетью
data['gen_synch'] = data['full_speed'] & (data['GTA1.DBinPU.P'] > 10)

In [ ]:
# номинальная нагрузка
data['vna_almost_open'] = data['gen_synch'] & (data['GTA1.DBinPU.Alvna'] > -8)

In [ ]:
# расчет изменения мощности за 1 мин
data['diff_P'] = data['GTA1.DBinPU.P'].diff(periods=2).shift(periods=-1).rolling(window='60s', center=True).mean()

In [ ]:
# увеличение мощности
data['increase_power'] = data['diff_P'] > 40

In [ ]:
# снижение мощности
data['decrease_power'] = data['diff_P'] < -40

In [ ]:
# расчет изменения положения ВНА
data['diff_Alvna'] = data['GTA1.DBinPU.Alvna'].diff(periods=2).shift(periods=-1).rolling(window='60s', center=True).mean()

In [ ]:
# открытие вна
data['increase_vna'] = data['diff_Alvna'] > 0.3

In [ ]:
# закрытие вна
data['decrease_vna'] = data['diff_Alvna'] < -0.3

In [ ]:
# расчет скорости изменения оборотов силовой турбины
data['diff_nst'] = data['GTA1.DBinPU.nst'].diff(periods=2).shift(periods=-1).rolling(window='60s', center=True).mean()

In [ ]:
# увеличение скорости вращения силовой турбины
data['increase_nst'] = data['diff_nst'] > 2

In [ ]:
# снижение скорости вращения силовой турбины
data['decrease_nst'] = data['diff_nst'] < -2

In [ ]:
# изменение скорости вращения турбокомпрессора
data['diff_ntk'] = data['GTA1.DBinPU.ntk'].diff(periods=2).shift(periods=-1)

In [ ]:
# увеличение скорости вращения турбокомпрессора
data['increase_ntk'] = data['diff_ntk'] > 50

In [ ]:
# снижение скорости вращения турбокомпрессора
data['decrease_ntk'] = data['diff_ntk'] < -50

In [ ]:
# подача топлива
data['fuel_off'] = data['GTA1.DBinPU.Aldi'] < 0.2

#### Классифицируем режимы

In [ ]:
data["full_power_mode"] = data['vna_almost_open'] & (~data['increase_vna']) & (~data['decrease_vna'])

In [ ]:
data["partial_power_mode"] = (~data['vna_almost_open']) & (~data['increase_vna']) & (~data['decrease_vna'])

In [ ]:
data["increas_power_mode"] = data['increase_vna'] & data['increase_power']

In [ ]:
data["decreas_power_mode"] = data['decrease_vna'] & data['decrease_power']

In [ ]:
data["start_up_mode"] = ((data['fuel_off'] & data['increase_ntk'])
                       | ((~data['fuel_off'] & data['increase_nst']))
                       | ((~data['increase_vna']) & data['increase_power']))

In [ ]:
data["shutdown_mode"] = ((data['fuel_off'] & data['decrease_ntk'])
                       | ((~data['fuel_off'] & data['decrease_nst']))
                       | ((~data['decrease_vna']) & data['decrease_power']))

In [ ]:
data["stopped_state_mode"] = data['GTA1.DBinPU.ntk'] < 10

#### Посмотрим на результат нашей разметки

In [ ]:
param1_dropdown = widgets.Dropdown(
    options=data.columns,
    description='Параметр1:',
    disabled=False,
    value=None
)

param2_dropdown = widgets.Dropdown(
    options=data.columns,
    description='Параметр2:',
    disabled=False,
    value=None
)

build_button = widgets.Button(description="Построить")

out = widgets.Output()
display(out)

@out.capture()
def build_button_eventhandler(btn):
    clear_output()

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    fig.add_traces(
        [
            {"x": data[param1_dropdown.value].index, "y": data[param1_dropdown.value].values, "name": param1_dropdown.value, "type": "scatter"},
            {"x": data[param2_dropdown.value].index, "y": data[param2_dropdown.value].values, "name": param2_dropdown.value, "type": "scatter"},
        ],
        rows=[1,2],
        cols=[1,1]
    )
    fig.update_layout(height=800, width=1400)
    display(param1_dropdown)
    display(param2_dropdown)
    display(build_button)
    display(fig)

build_button.on_click(build_button_eventhandler)

with out:
    display(param1_dropdown)
    display(param2_dropdown)
    display(build_button)

Output()

### Сохранение подготовленных данных

In [ ]:
df.to_parquet("data.gzip", compression='gzip')

In [ ]:
# df_start_up_mode = df[df["start_up_mode"]]
# df_start_up_mode["start_up_mode_#"] = (df_start_up_mode.index.to_series().diff() > pd.Timedelta('10s')).cumsum()

# for i in df_start_up_mode["start_up_mode_#"].unique():
#     temp_df = df_start_up_mode[df_start_up_mode['start_up_mode_#'] == i]
#     if temp_df.shape[0] > 1:
#         print(f'{temp_df.index[0]} --- {temp_df.index[-1]}')

In [ ]:
# df_increas_power_mode = df[df["increas_power_mode"]]
# df_increas_power_mode["increas_power_mode_#"] = (df_increas_power_mode.index.to_series().diff() > pd.Timedelta('10s')).cumsum()

# for i in df_increas_power_mode["increas_power_mode_#"].unique():
#     temp_df = df_increas_power_mode[df_increas_power_mode['increas_power_mode_#'] == i]
#     if temp_df.shape[0] > 1:
#         print(f'{temp_df.index[0]} --- {temp_df.index[-1]}')